# Middleware: Human In The Loop
<img src="../python/assets/LC_HITL.png" width="300">



First, we'll connect to our SQLite database that contains employee and music store data.


In [1]:
import { SqlDatabase } from "@langchain/classic/sql_db";
import { DataSource } from "typeorm";

const datasource = new DataSource({
    type: "sqlite",
    database: "./Chinook.db", // Replace with the link to your database
});
const db = await SqlDatabase.fromDataSourceParams({
    appDataSource: datasource,
});

Now we'll create a tool that allows the agent to execute SQL queries. This is what we'll require approval for.


In [2]:
import { tool } from "langchain";
import { z } from "zod";

export const executeSQL = tool(({ query }) => {
    return db.run(query)
}, {
    name: "execute_sql",
    description: "Execute a SQLite command and return results.",
    schema: z.object({ query: z.string() })
})

We'll define a system prompt that guides the agent to be careful and methodical when working with the database.


In [3]:
const SYSTEM = `You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool \`execute_sql\` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
- If the database is offline, ask user to try again later without further comment.`

Next, we'll create an agent with **Human-in-the-Loop** middleware. Every time the agent wants to execute SQL, it will pause and wait for human approval.


In [4]:
import * as setup from "./setup.ts";
import { createAgent, humanInTheLoopMiddleware } from "langchain";
import { MemorySaver } from "@langchain/langgraph";

const checkpointer = new MemorySaver();
const agent = createAgent({
    model: "anthropic:claude-sonnet-4-5-20250929",
    tools: [executeSQL],
    systemPrompt: SYSTEM,
    checkpointer,
    middleware: [humanInTheLoopMiddleware({
        interruptOn: {
            execute_sql: {
                allowAccept: true,
                allowRespond: true,
                allowEdit: false // No editing allowed
            }
        },
        descriptionPrefix: "Tool execution pending approval"
    })]
});

Let's test it! We'll ask about employees. The agent will pause at each SQL query, and we'll approve them by resuming with `type: "accept"`.


In [5]:
import { HumanMessage } from "langchain";
import { Command } from "@langchain/langgraph";

const question = new HumanMessage("What are the names of all the employees?");
const config = { configurable: { thread_id: "1" } } 

let result = await agent.invoke({
    messages: [question]
}, config)

while ('__interrupt__' in result) {
    console.log("\x1b[31m" + "-".repeat(80) + "\x1b[0m");
    console.log("\x1b[31m" + `Interrupt:${result['__interrupt__'][0].value[0]['description']}` + "\x1b[0m");
    console.log("\x1b[31m" + "-".repeat(80) + "\x1b[0m");

    result = await agent.invoke(
        new Command({
            resume: [{ type: "accept" }]
            // resume: [{type: "response", args: "the database is offline."}]
        }),
        config
    )
}

result.messages.at(-1).content;

--------------------------------------------------------------------------------
Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {
  "query": "SELECT name FROM employees LIMIT 5"
}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {
  "query": "SELECT name FROM sqlite_master WHERE type='table'"
}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {
  "query": "SELECT FirstName, LastName FROM Employee LIMIT 5"
}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Interrupt:Tool execution requires approval



"Here are the names of all 8 employees:\n" +
  "\n" +
  "1. Andrew Adams\n" +
  "2. Nancy Edwards\n" +
  "3. Jane Peacock\n" +
  "4. Margaret Park\n" +
  "5. Steve Johnson\n" +
  "6. Michael Mitchell\n" +
  "7. Robert King\n" +
  "8. Laura Callahan"